In [1]:
import numpy as np
import tensorflow as tf
import time
# import random

/home/sogol/anaconda3/lib/python3.7/site-packages/tensorflow-2.0.0b1-py3.7-linux-x86_64.egg/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sogol/anaconda3/lib/python3.7/site-packages/tensorflow-2.0.0b1-py3.7-linux-x86_64.egg/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sogol/anaconda3/lib/python3.7/site-packages/tensorflow-2.0.0b1-py3.7-linux-x86_64.egg/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint

In [2]:
# gpu_available = tf.test.is_gpu_available()
# is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
# is_cuda_gpu_min_3 = tf.test.is_gpu_available(True, (3,0))
# is_cuda_gpu_min_3

In [3]:
train = np.load('../data/freebase15k_numpy/train.npy')
validation = np.load('../data/freebase15k_numpy/validation.npy')
test = np.load('../data/freebase15k_numpy/test.npy')

entities = np.load('../data/freebase15k_numpy/entities.npy')
relations = np.load('../data/freebase15k_numpy/relations.npy')

train = tf.convert_to_tensor(train.astype(dtype=np.int16), dtype=tf.int32)
validation = tf.convert_to_tensor(validation.astype(dtype=np.int16), dtype=tf.int16)
test = tf.convert_to_tensor(test.astype(dtype=np.int16), dtype=tf.int16)
relations = tf.convert_to_tensor(relations.astype(dtype=np.int16), dtype=tf.int16)
entities = tf.convert_to_tensor(entities.astype(dtype=np.float64), dtype=tf.float64)

2021-12-17 18:38:09.798353: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-12-17 18:38:09.832189: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-12-17 18:38:09.832589: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5615bce0d070 executing computations on platform Host. Devices:
2021-12-17 18:38:09.832603: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>


In [4]:
iteration = tf.constant(500)
dim = tf.constant(100)
kns = tf.constant(5)
alpha = tf.Variable(0.000001, dtype = tf.float64)
beta = tf.Variable(0.0000001, dtype = tf.float64)
z = tf.constant(0.5, dtype = tf.float64)
nn1 = tf.random.uniform( shape=(entities.shape[0], dim),
      minval=tf.math.truediv(z , tf.cast(dim,tf.float64),'minval'), 
      maxval=tf.math.truediv(z , tf.cast(dim,tf.float64)),
      dtype=tf.dtypes.float64, seed=None, name='nn1')
      
nn0 =tf.random.uniform(shape=(entities.shape[0], dim), 
      minval=tf.math.truediv(z , tf.cast(dim,tf.float64)), 
      maxval=tf.math.truediv(z , tf.cast(dim,tf.float64)),
      dtype=tf.dtypes.float64, seed=None, name='nn0')
nn2 = tf.random.uniform(shape=(relations.shape[0], dim), 
      minval=tf.math.truediv(z , tf.cast(dim,tf.float64)), 
      maxval=tf.math.truediv(z , tf.cast(dim,tf.float64)),
      dtype=tf.dtypes.float64, seed=None, name='nn2')

In [5]:
def calculateRank(triple):   
    selectedEntities = entities[:,0]
    if(len((np.where((train[:,1] == triple[1]) & (train[:,2] == triple[2]))[0]))> 0):
        selectedEntities = np.where(np.isin(selectedEntities , train[np.where((train[:,1] ==triple[1]) & (train[:,2] ==triple[2]))][0], invert =True ))[0]
    if(len((np.where((test[:,1] == triple[1]) & (test[:,2] == triple[2]))[0]))> 0):
        selectedEntities = np.where(np.isin(selectedEntities , test[np.where((test[:,1] ==triple[1]) & (test[:,2] ==triple[2]))][0], invert =True ))[0]
    if(len((np.where((validation[:,1] == triple[1]) & (validation[:,2] == triple[2]))[0]))> 0):
        selectedEntities = np.where(np.isin(selectedEntities , validation[np.where((validation[:,1] ==triple[1]) & (validation[:,2] ==triple[2]))][0], invert =True ))[0]
    couraptedH = np.empty((selectedEntities.shape[0],3))
    couraptedH[:,0] =selectedEntities
    couraptedH[:,1] =triple[1]
    couraptedH[:,2] =triple[2]
    
    selectedEntities = entities[:,0]
    if(len((np.where((train[:,0] == triple[0]) & (train[:,2] == triple[2]))[0]))> 0):
        selectedEntities = np.where(np.isin(selectedEntities , train[np.where((train[:,0] == triple[0]) & (train[:,2] == triple[2]))][0], invert =True ))[0]
    if(len((np.where((test[:,0] == triple[0]) & (test[:,2] == triple[2]))[0]))> 0):
        selectedEntities = np.where(np.isin(selectedEntities , test[np.where((test[:,0] ==triple[0]) & (test[:,2] ==triple[2]))][0], invert =True ))[0]
    if(len((np.where((validation[:,0] == triple[0]) & (validation[:,2] == triple[2]))[0]))> 0):
        selectedEntities = np.where(np.isin(selectedEntities , validation[np.where((validation[:,0] ==triple[0]) & (validation[:,2] ==triple[2]))][0], invert =True ))[0]
    couraptedT = np.empty((selectedEntities.shape[0],3))
    couraptedT[:,0] = triple[0]
    couraptedT[:,1] = selectedEntities
    couraptedT[:,2] =triple[2]
    courapted = np.concatenate((couraptedH, couraptedT), axis=0)
    courapted = courapted.astype(np.int16)
    courapted = np.vstack([courapted, triple])
    
    p =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(nn1[courapted[:,1]],tf.transpose(nn0[courapted[:,0]] + nn2[triple[2]] ),axes=1 )))
    
    courapted =np.concatenate((courapted, p[:, None]), axis=1)
    courapted = courapted[np.argsort(courapted[:, -3])]
    return np.where((courapted[:,0] == triple[0]) & (courapted[:,1] == triple[1]))[0][0]

In [6]:
def mrr(ranks):
    inverse = []
    for rank in ranks:
        inverse.append(float(1)/rank)
    summ = sum(inverse)
    return float(1)/len(inverse) * summ

# 483142    

In [7]:
start = time.time()
t_row = train.shape[0]
for x in range(0 , iteration):
    # tf.random.shuffle(train, seed=None, name=None)
    total_samples  = tf.gather( tf.random.categorical([tf.gather(entities, 2, axis=1)], tf.math.multiply(kns,t_row) , dtype=None, seed=None),0)
    for tIndex in range(0 , t_row):
        triple = tf.gather(train, tIndex)
        _head_index = tf.gather(triple,0)
        _relation_index = tf.gather(triple,2)
        _vHead = tf.gather(nn0,_head_index)
        _vRel = tf.gather(nn2,_relation_index)
        samples = [tf.slice(total_samples,begin=[tf.math.multiply(tIndex, kns)],size=[kns])]
        # print(samples)
        # print( tf.random.categorical([tf.gather(entities, 2, axis=1)], kns , dtype=None, seed=None))
        paddings = tf.constant([[0, 1,], [0, 0]])
        samples = tf.pad(samples, paddings, constant_values=0)
        samples =  tf.cast(samples, tf.int32)         
        samples = tf.transpose(tf.concat([samples, [[tf.gather(triple,1)],[1]]], 1))
        indices = tf.gather(samples, 0, axis=1)
        _nn1_samples = tf.gather(nn1, indices)
        _sigmoid =tf.math.sigmoid(tf.tensordot( _nn1_samples , tf.transpose(tf.math.add(_vHead , _vRel)) , axes=1))
        cost = tf.math.subtract(tf.cast(tf.gather(samples, 1, axis=1), tf.float64) , _sigmoid)
        g = tf.math.multiply(alpha , cost)
        g1 = tf.math.multiply(beta , cost)
        _nn1_samples = tf.math.add(tf.math.multiply(tf.gather(nn1, indices) , tf.reshape(g, (6, 1))) , _nn1_samples)
        nn1 = tf.tensor_scatter_nd_update(nn1,tf.expand_dims(indices, 1),_nn1_samples)

        _nn2_sample = tf.math.add_n([tf.math.reduce_sum(tf.math.multiply(_vRel , tf.reshape(g1, (g1.shape[0], 1))), axis=0, keepdims=False)  , _vRel, tf.math.reduce_sum(_nn1_samples, axis=0, keepdims=False)])
        indices = tf.constant(_relation_index)
        nn2 = tf.tensor_scatter_nd_update(nn2,tf.expand_dims([indices], 1), tf.reshape(_nn2_sample,(1, _nn2_sample.shape[0])))
  
        indices = tf.constant(_head_index)
        _nn0_sample = tf.math.add(tf.math.reduce_sum(tf.math.multiply(_vHead , tf.reshape(g, (g.shape[0], 1))), axis=0, keepdims=False)  , _nn2_sample)
        nn0 = tf.tensor_scatter_nd_update(nn0,tf.expand_dims([indices], 1), tf.reshape(_nn0_sample,(1, _nn0_sample.shape[0])))
    print(time.time() - start)        
    # print('total one sample calc time =>  ' , time.time() - start)       
    # if x % 10 == 0 and x > 20:
# _mrrList = []
# # for triple in validation:
# _mrrList.append(calculateRank(validation[0,:]))
# print('total one sample calc time =>  ' , time.time() - start) 
# 1696.807552099228

3398.960925579071
6424.747095108032


KeyboardInterrupt: 